In [122]:
import requests, time
import pickle
from datetime import datetime, date, timedelta
import pandas as pd
from pandas import DataFrame as df
import matplotlib.pyplot as plt
import numpy as np

NEWS_URL = "https://min-api.cryptocompare.com/data/v2/news/?lang=EN"
NEWS_CATEGORY_URL = "https://min-api.cryptocompare.com/data/news/categories"
LIMIT_QUERY = 2000

In [23]:
'''
currency : "BTC", "ETH", "BCH", ...
'''
def news_request(category, lTs = int(datetime.now().timestamp())):
    params = {
        "categories" : category, 
        "lTs" : lTs
    }
    req = requests.get(NEWS_URL, params = params).json()
    return req["Data"]

In [24]:
def date_to_timestamp(date_str):
    date = datetime.strptime(date_str, "%Y-%m-%d")
    end_of_date = date + timedelta(days = 1) - timedelta(seconds = 1)
    return end_of_date

In [25]:
def is_same_date(a, b):
    return a.year == b.year and a.month == b.month and a.day == b.day

In [26]:
def get_news_data(category, news_per_date, start_date, end_date):
    result = []

    last_time_from = date_to_timestamp(end_date)
    
    while last_time_from >= datetime.strptime(start_date, "%Y-%m-%d"):
        data = news_request(category, int(datetime.timestamp(last_time_from)))
        i = 0
        while last_time_from >= datetime.strptime(start_date, "%Y-%m-%d"):
            while i< len(data) and is_same_date(date.fromtimestamp(data[i]['published_on']), last_time_from):
                result.append(data[i])
                i += 1
            if i >= len(data):
                last_time_from -= timedelta(days = 1)
                break
            else:
                last_time_from = datetime.fromtimestamp(data[i]['published_on'])
            
    return result

In [27]:
def to_dataframe(list_data):
    add_date = [data | {'date': date.fromtimestamp(data['published_on'])} for data in list_data]
    try:
        dataframe = df(add_date)[['date', 'id', 'published_on', 'title', 'body', 'url', 'imageurl', 'tags', 'categories']]
        return dataframe
    except:
        return df({})

In [28]:
def get_news_data_df(category, news_per_date, start_date, end_date):
    new_data = get_news_data(category, news_per_date, start_date, end_date)    
    return to_dataframe(new_data)

In [30]:
def save_df(dataframe, category,year_str, output_dir = "./data/news/"):
    name = "news_{}.csv".format(year_str)
    dataframe.to_csv(output_dir + "/" + name)

In [31]:
def get_and_save_news_data(category, news_per_date = 10, \
        start_date = datetime.now().date().strftime("%Y-%m-%d"), \
        end_date = datetime.now().date().strftime("%Y-%m-%d"), \
        output_dir = "./data"):
    dataframe = get_news_data_df(category, news_per_date, start_date, end_date)
    save_df(dataframe, category, start_date[:4], output_dir)
    print("done")

In [33]:
def get_categories():
    req = requests.get(NEWS_CATEGORY_URL).json()
    result = [value['categoryName'] for value in req]
    return result

In [34]:
categories = get_categories()
for year in range(2022, 2012, -1):
    get_and_save_news_data(categories, None, \
        "{}-1-1".format(year), "{}-12-31".format(year),
        output_dir = "./data/news")

done
done
done
done
done
done
done
done
done
done
